In [1]:
import pandas as pd
import re
import numpy as np
import math
from datetime import datetime, timedelta
import os
import time
import glob

pd.set_option('display.expand_frame_repr', False)
#pd.set_option('display.max_colwidth', -1)
# Reduce decimal points to 2
pd.options.display.float_format = '{:,.4f}'.format

In [2]:
#set my desired working directory
os.chdir("/Users/ariel/Dropbox/PhD/Michael CSR/Analysis")

In [8]:
#load time series of fund sustanability
fund_sust=pd.read_csv('Jupyter/fund_sust.csv')
fund_sust=fund_sust.drop(columns=['Unnamed: 0'])
print(fund_sust.head())

                Name fund_cusip            Global Category  portfolio_score_date  portfolio_score  crsp_fundno    crsp_portno
0  Invesco Charter A   00141310  US Equity Large Cap Blend                201209          51.4100   2,899.0000 1,022,003.0000
1  Invesco Charter A   00141310  US Equity Large Cap Blend                201212          50.9000   2,899.0000 1,022,003.0000
2  Invesco Charter A   00141310  US Equity Large Cap Blend                201303          51.0100   2,899.0000 1,022,003.0000
3  Invesco Charter A   00141310  US Equity Large Cap Blend                201306          51.4400   2,899.0000 1,022,003.0000
4  Invesco Charter A   00141310  US Equity Large Cap Blend                201309          49.9600   2,899.0000 1,022,003.0000


In [ ]:
#load data file
holding_data=pd.read_csv('Jupyter/holding_data.csv') #save
print(holding_data.head())